In [1]:
from pyspark.sql import SparkSession
import sys, os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', '..')))
from src.processing.process_data import format_date, format_bdi, format_spec

In [2]:
spark = SparkSession\
    .builder\
    .master('local[*]')\
    .appName('load')\
    .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/11/08 16:26:20 WARN Utils: Your hostname, Giordano, resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/11/08 16:26:20 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/08 16:26:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df = spark\
    .read\
    .parquet('../../datalake/Bovespa')

In [4]:
df = format_date(df)
df = format_bdi(df)
df = format_spec(df)
df.show(5)

25/11/08 16:26:33 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------------+------------+--------+----------------+-----------+------------+-------------+---------------------------+--------+-------------+----------+----------+----------+----------------+-------------------------+------------------------+----------------+-----------------------+-----------------------+-------------------------------------------------+-------------------------------------------------------------+----------------------------------------------------+--------------------------+-------------------------------------------------------------------------+--------------------+-------------------+----+-----+-------------------+-------------------+--------------------+
|Register Type|Trading Date|BDI Code|Negociation Code|Market Type|  Trade Name|Specification|Forward Market Term In Days|Currency|Opening Price|Max. Price|Min. Price|Mean Price|Last Trade Price|Best Purshase Order Price|Best Purshase Sale Price|Numbor Of Trades|Number Of Traded Stocks|Volume Of Traded Stocks|

In [6]:
df\
    .write\
    .mode('overwrite')\
    .partitionBy('Year', 'Month')\
    .parquet('../../datalake/serving/Bovespa')